# Richter's Predictor Project

In this project, I will be working on predicting the levels of damage to a building (1 - low, 2 - medium, 3 - high) caused by the 2015 Gorkha Earthquake in Nepal. 

In [19]:
import pandas as pd
import sklearn as sk

In [67]:
# reading the training features dataset and printing it out
train_set_features = pd.read_excel("train_Features.xlsx")

# seperate the column names based on the comma
new_columns = []
new_columns = train_set_features.columns[0].split(',')


#seperate the row names on the comma
values = []
for col in train_set_features.columns:
    for row in train_set_features[col]:
        if isinstance(row, str):  # Check if the value is a string
            values.append(row.split(','))  # Split string values by commas
        else:
            values.append([row])  # If it's not a string, just keep it as is

clean_train_set_features = pd.DataFrame(values,columns = new_columns) # creates new clean dataframe
clean_train_set_features.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,0,0,0,0
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,0,0,0,0,0


In [68]:
clean_train_set_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   building_id                             260601 non-null  object
 1   geo_level_1_id                          260601 non-null  object
 2   geo_level_2_id                          260601 non-null  object
 3   geo_level_3_id                          260601 non-null  object
 4   count_floors_pre_eq                     260601 non-null  object
 5   age                                     260601 non-null  object
 6   area_percentage                         260601 non-null  object
 7   height_percentage                       260601 non-null  object
 8   land_surface_condition                  260601 non-null  object
 9   foundation_type                         260601 non-null  object
 10  roof_type                               260601 non-null 